# Exercise: Flood Analysis in Opole (2024) using Sentinel-1 SAR Data

In this exercise, you will compare radar backscatter from Sentinel-1 before and after the 2024 flood in Opole, Poland.

**Objective:**
- Load Sentinel-1 VV data before and after the flood
- Convert radar intensity to decibels (dB)
- Calculate the difference between the two dates
- Identify areas with a significant decrease in VV (possible flood zones)

## Step 1: Import Earth Engine and geemap

In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='ee-kacperkrempa01')
aoi = ee.Geometry.Point([17.931, 50.675])

## Step 2: Load Sentinel-1 VV images (before and after flood)

In [2]:
# TODO: Load Sentinel-1 collection for before and after flood
# Use filterBounds, filterDate, and select 'VV' band
# Apply .mean() to get average image from each period

before = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate('2024-09-12', '2024-09-25') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')\
    .mean()

after = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate('2024-10-15', '2024-10-30') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')\
    .mean()

## Step 3: Convert to dB scale

In [3]:
# TODO: Use .log10().multiply(10) to convert before/after images to dB
before_db = before.log10().multiply(10)
after_db = after.log10().multiply(10)

## Step 4: Calculate difference and threshold potential flood areas

In [4]:
# TODO: Subtract after_dB from before_dB
# Create flood mask: pixels where difference > 3 dB

difference = before_db.subtract(after_db)
flood_mask = difference.gt(3)

## Step 5: Visualize the results

In [5]:
# TODO: Use Map.addLayer() to show before, after, difference, and flood mask
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
Map.addLayer(before_db, {'min': -25, 'max': 0}, 'Before Flood (dB)')
Map.addLayer(after_db, {'min': -25, 'max': 0}, 'After Flood (dB)')
Map.addLayer(difference, {'min': -10, 'max': 10}, 'Difference (dB)')
Map.addLayer(flood_mask.updateMask(flood_mask), {'palette': ['blue']}, 'Flood Mask')
Map

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…